#**Export a video as MP4**
- Enter your favourite character
- Gradio application will generate story for you and related video

###**Install Dependencies**

In [1]:
!pip install gradio numpy diffusers openai

###**Implementation**
We require a GPU to generate video clips

Menu Bar --> Runtime --> Change Runtime --> T4 GPU

We will use diffusion model (damo-vilab/text-to-video-ms-1.7b) from Huggingface

###**Retrive API key from Secrets and Set as an ENV**

In [2]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
# Set OPENAI_API_KEY as an ENV
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [4]:
from openai import OpenAI
client = OpenAI()

###**Python Code**

In [5]:
import gradio as gr
import torch
import numpy as np
from PIL import Image
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
import gc

# Load text-to-video model
pipe = DiffusionPipeline.from_pretrained(
    "damo-vilab/text-to-video-ms-1.7b",
    torch_dtype=torch.float16,
    variant="fp16",
    low_cpu_mem_usage=True
)
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe.to(device)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

def generate_story(character_name):
    response = client.responses.create(
        model="gpt-4o-mini",
        input=[
            {"role": "user", "content": f"Write a short, 3-sentence story about {character_name} that is fun and visual."}
        ]
    )
    story = response.output_text
    return story

def generate_video(prompt):
    output = pipe(prompt, num_inference_steps=20)
    frames_array = output.frames

    if isinstance(frames_array, np.ndarray) and frames_array.ndim == 5:
        frames_array = frames_array[0]
    else:
        raise ValueError("Unexpected format for frames")

    rgb_frames = [(frame * 255).clip(0, 255).astype(np.uint8) for frame in frames_array]
    pil_frames = [Image.fromarray(frame) for frame in rgb_frames]

    video_path = "generated_story_video.mp4"
    export_to_video(pil_frames, output_video_path=video_path)

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return video_path

def process(character_name):
    story = generate_story(character_name)
    video_path = generate_video(story)
    return story, video_path

with gr.Blocks() as demo:
    gr.Markdown("""# Story to Video Generator\nEnter a character and get a short story and animated video.""")
    name_input = gr.Textbox(label="Character Name")
    generate_btn = gr.Button("Generate Story and Video")
    story_output = gr.Textbox(label="Generated Story")
    video_output = gr.File(label="Download Video (MP4)")

    generate_btn.click(fn=process, inputs=name_input, outputs=[story_output, video_output])

if __name__ == "__main__":
    demo.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c3f9af24385b21da10.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
